In [3]:
%matplotlib

import math
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

class Laguerre:
    def __init__(self, function, alpha, beta, N, T):
        self.function = function
        self.alpha = alpha
        self.beta = beta
        self.N = N
        self.T = T
        self.processes()

    @property
    def function(self):
        return self._function

    @function.setter
    def function(self, value):
        self._function = value

    @property
    def alpha(self):
        return self._alpha

    @alpha.setter
    def alpha(self, value):
        self._alpha = value

    @property
    def beta(self):
        return self._beta

    @beta.setter
    def beta(self, value):
        self._beta = value
        
    @property
    def N(self):
        return self._N

    @N.setter
    def N(self, value):
        self._N = value
        
    @property
    def T(self):
        return self._T

    @T.setter
    def T(self, value):
        self._T = value

    @property
    def sigma(self):
        return self._alpha + self._beta

    def __str__(self):
        return f'Alpha:{self.alpha} \nBeta:{self.beta} \nSigma:{self.sigma}'

    def fx(self, x):
        return float(eval(self.function))

    def lk(self, t, k):
        mas_k = []     
        for i in range(0, k+1):
            if i == 0:
                mas_k.append(np.sqrt(self.sigma) * np.exp((-self.beta / 2) * t))
            elif i == 1:
                mas_k.append(np.sqrt(self.sigma) * (1 - self.sigma * t) * np.exp((-self.beta / 2) * t))
            else: 
                mas_k.append(((2 * i - 1 - t) / i) * mas_k[i-1] - ((i - 1) / i) * mas_k[i-2])
        result = mas_k[len(mas_k)-1]
        del mas_k
        return result
    
    def f(self, t, k):
        return self.fx(t) * self.lk(t, k) * np.exp(-(self.sigma - self.beta) * t)

    def sum_rectangle(self, n, right, left, k):
        smt_value = n
        all_sum = 0
        add_h = (right - left) / n
        while smt_value >= 1:
            all_sum += self.f(left + ((smt_value - 0.5) * add_h), k)
            smt_value -= 1
        del smt_value, add_h
        return all_sum

    def integral(self, right, left, k):
        h = right - left
        value1_rct = h * self.f(left + h / 2, k)
        value2_rct = (h / 2) * self.sum_rectangle(2, right, left, k)
        n = 4
        coef = h / (2 * n)
        while np.abs((value1_rct - value2_rct) / value2_rct) > 0.01:
            value1_rct = value2_rct
            value2_rct = coef * self.sum_rectangle(n, right, left, k)
            n *= 2
            coef = h / n
        del h, value1_rct, n, coef
        return value2_rct

    def processes(self):
        try:
            add_func = ''
            for i in range(len(self.function)):
                if self.function[i] == '^':
                    add_func += '**'
                else:
                    add_func += self.function[i]
            self.function = add_func
            del add_func

            step = 0.2
            appr = np.exp(-10)
            
            right = 0
            while np.exp(-self.alpha * right) >= appr:
                right += 0.5
            del appr    
                
            left = 0
            
            fk = []
            for i in range(0, self.N):
                fk.append(self.integral(right, left, i))
            del left
            
            print('Інтеграл:')
            [print(i) for i in fk]
            hs = []
            for t_i in np.arange(0, self.T, step):
                sum_k = 0
                for k_i in fk:
                    sum_k += k_i * self.lk(t_i, fk.index(k_i))
                hs.append(sum_k)
            
            del fk
            
            xs = [i for i in np.arange(0, self.T, step)]
            ys = [self.fx(i) for i in np.arange(0, self.T, step)]
            
            del step
            
            fig, ax = plt.subplots()
            plt.axhline(color="black")
            plt.axvline(color="black")
            plt.plot(xs,ys)
            plt.plot(xs,hs)
            
            del xs, ys, hs
            
            fig.canvas.set_window_title('Апроксимацiя функцiями Лагерра - графік')
            plt.grid()
            plt.show(block=False)
        except:
            print('Помилка: перевірте введені дані.')

def chooseParametrs(Функція, Альфа, Бета, N, T):
    temp = Laguerre(Функція, float(Альфа), float(Бета), int(N), float(T))
    del temp

print('Введіть необхідні параметри:')
interact_manual(chooseParametrs, Функція='', Альфа='', Бета='', N='', T='');

Using matplotlib backend: Qt5Agg
Введіть необхідні параметри:


interactive(children=(Text(value='', description='Функція'), Text(value='', description='Альфа'), Text(value='…